# GDELT Bitcoin 기사 수집 + Tone 분석 + 가격/지수 병합

**기간**: 2025-11-23 ~ 2026-01-14 (UTC)  
**목표**:
1. GDELT DOC API로 Bitcoin 기사 수집 (시간당 최대 10건)
2. GKG V1.5TONE 데이터를 URL 기준으로 조인
3. BTC 종가 및 Fear & Greed Index와 병합

**필요 라이브러리**:
```bash
pip install pandas requests
```

## 셀 1: Bitcoin 기사 수집 + GKG Tone 조인 + Raw 저장

**작업 내용**:
- GDELT DOC API로 시간별 Bitcoin 기사 수집 (최대 10건/시간)
- GKG 일자별 ZIP에서 V1.5TONE 추출 및 매핑
- URL 기준 조인 → tone 있는 기사만 유지
- `./data/raw/251123-260114-news.csv` 저장

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
GDELT DOC API로 Bitcoin 기사 수집 + GKG Tone 조인
"""

import logging
import os
import zipfile
from datetime import datetime, timedelta, timezone
from io import BytesIO, TextIOWrapper
from pathlib import Path
from time import sleep

import pandas as pd
import requests

# 로깅 설정
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# ====== 설정 ======
START_DATE = datetime(2025, 11, 23, 0, 0, 0, tzinfo=timezone.utc)
END_DATE = datetime(2026, 1, 14, 23, 59, 59, tzinfo=timezone.utc)
OUTPUT_PATH = './data/raw/251123-260114-news.csv'
MAX_ARTICLES_PER_HOUR = 10
TIMEOUT = 30
RETRY_COUNT = 3

# ====== 1. GDELT DOC API로 시간별 Bitcoin 기사 수집 ======
def fetch_gdelt_doc_hourly(start_dt, end_dt, query="bitcoin", max_per_hour=10):
    """
    시간별로 GDELT DOC API를 호출하여 기사 목록 수집
    
    Args:
        start_dt: 시작 datetime (UTC)
        end_dt: 종료 datetime (UTC)
        query: 검색 키워드
        max_per_hour: 시간당 최대 수집 건수
    
    Returns:
        list of dict: 수집된 기사 목록
    """
    base_url = "https://api.gdeltproject.org/api/v2/doc/doc"
    articles = []
    
    current = start_dt
    hour_count = 0
    
    while current < end_dt:
        next_hour = current + timedelta(hours=1)
        
        # GDELT 날짜 형식: YYYYMMDDHHMMSS
        start_str = current.strftime('%Y%m%d%H%M%S')
        end_str = next_hour.strftime('%Y%m%d%H%M%S')
        
        params = {
            'query': query,
            'mode': 'ArtList',
            'format': 'json',
            'startdatetime': start_str,
            'enddatetime': end_str,
            'maxrecords': max_per_hour
        }
        
        try:
            response = requests.get(base_url, params=params, timeout=TIMEOUT)
            response.raise_for_status()
            data = response.json()
            
            if 'articles' in data:
                hour_articles = data['articles'][:max_per_hour]
                articles.extend(hour_articles)
                logger.info(f"[{start_str}] 수집: {len(hour_articles)}건")
            else:
                logger.warning(f"[{start_str}] 'articles' 필드 없음")
            
            hour_count += 1
            
            # API 과부하 방지 (100시간마다 쉬기)
            if hour_count % 100 == 0:
                logger.info(f"누적 {hour_count}시간 수집 완료, 잠시 대기...")
                sleep(2)
        
        except Exception as e:
            logger.error(f"[{start_str}] DOC API 호출 실패: {e}")
        
        current = next_hour
    
    logger.info(f"총 {len(articles)}건의 기사 수집 완료")
    return articles


# ====== 2. GKG 일자별 ZIP 다운로드 및 V1.5TONE 파싱 ======
def download_gkg_zip(date_str, retry=3):
    """
    GDELT GKG 일자별 ZIP 파일 다운로드
    
    Args:
        date_str: YYYYMMDD 형식 날짜
        retry: 재시도 횟수
    
    Returns:
        BytesIO or None
    """
    # GKG 2.0 영어 파일 URL
    url = f"http://data.gdeltproject.org/gdeltv2/{date_str}.gkg.csv.zip"
    
    for attempt in range(retry):
        try:
            response = requests.get(url, timeout=TIMEOUT)
            response.raise_for_status()
            return BytesIO(response.content)
        except Exception as e:
            logger.warning(f"[{date_str}] GKG ZIP 다운로드 실패 (시도 {attempt+1}/{retry}): {e}")
            if attempt < retry - 1:
                sleep(2)
    
    logger.error(f"[{date_str}] GKG ZIP 다운로드 최종 실패")
    return None


def parse_gkg_tone(date_str):
    """
    GKG ZIP에서 V1.5TONE 추출 (URL -> tone_mean, tone_pos_share)
    
    Args:
        date_str: YYYYMMDD 형식 날짜
    
    Returns:
        dict: {url: {'tone_mean': float, 'tone_pos_share': float}}
    """
    zip_data = download_gkg_zip(date_str)
    if not zip_data:
        return {}
    
    tone_map = {}
    
    try:
        with zipfile.ZipFile(zip_data) as zf:
            csv_name = f"{date_str}.gkg.csv"
            
            with zf.open(csv_name) as f:
                # GKG 2.0 컬럼: 탭 구분, 컬럼 4 = DocumentIdentifier, 컬럼 15 = V15Tone
                text_file = TextIOWrapper(f, encoding='utf-8', errors='ignore')
                
                for line in text_file:
                    parts = line.strip().split('\t')
                    
                    if len(parts) < 16:
                        continue
                    
                    url = parts[4]  # DocumentIdentifier
                    tone_str = parts[15]  # V15Tone
                    
                    if not url or not tone_str:
                        continue
                    
                    # V15Tone 형식: Tone,PositiveScore,NegativeScore,Polarity,ActivityReferenceDensity,SelfGroupReferenceDensity
                    tone_parts = tone_str.split(',')
                    
                    if len(tone_parts) < 3:
                        continue
                    
                    try:
                        tone_mean = float(tone_parts[0])
                        positive = float(tone_parts[1])
                        negative = float(tone_parts[2])
                        
                        # tone_pos_share 계산 (0분모 방지)
                        total = positive + negative
                        tone_pos_share = positive / total if total > 0 else 0.5
                        
                        tone_map[url] = {
                            'tone_mean': tone_mean,
                            'tone_pos_share': tone_pos_share
                        }
                    
                    except ValueError:
                        continue
        
        logger.info(f"[{date_str}] GKG Tone 파싱: {len(tone_map)}개 URL")
    
    except Exception as e:
        logger.error(f"[{date_str}] GKG 파싱 실패: {e}")
    
    return tone_map


def build_tone_mapping(start_dt, end_dt):
    """
    기간 내 모든 GKG 데이터에서 URL -> Tone 매핑 구축
    
    Args:
        start_dt: 시작 datetime
        end_dt: 종료 datetime
    
    Returns:
        dict: {url: {'tone_mean': float, 'tone_pos_share': float}}
    """
    tone_mapping = {}
    
    current_date = start_dt.date()
    end_date = end_dt.date()
    
    while current_date <= end_date:
        date_str = current_date.strftime('%Y%m%d')
        daily_tone = parse_gkg_tone(date_str)
        tone_mapping.update(daily_tone)
        
        current_date += timedelta(days=1)
    
    logger.info(f"총 {len(tone_mapping)}개의 URL에 대한 Tone 매핑 완료")
    return tone_mapping


# ====== 3. DOC 결과와 Tone 조인 ======
def join_articles_with_tone(articles, tone_mapping):
    """
    기사 목록에 Tone 데이터를 조인하고, tone이 있는 기사만 유지
    
    Args:
        articles: DOC API 결과 (list of dict)
        tone_mapping: URL -> Tone 매핑 (dict)
    
    Returns:
        pd.DataFrame
    """
    df = pd.DataFrame(articles)
    
    if df.empty:
        logger.warning("기사 데이터가 비어있음")
        return pd.DataFrame()
    
    # Tone 데이터 조인
    df['tone_mean'] = df['url'].map(lambda x: tone_mapping.get(x, {}).get('tone_mean'))
    df['tone_pos_share'] = df['url'].map(lambda x: tone_mapping.get(x, {}).get('tone_pos_share'))
    
    # tone_mean이 있는 행만 유지
    before_count = len(df)
    df = df.dropna(subset=['tone_mean'])
    after_count = len(df)
    
    match_rate = (after_count / before_count * 100) if before_count > 0 else 0
    logger.info(f"Tone 매칭: {after_count}/{before_count}건 ({match_rate:.1f}%)")
    
    return df


# ====== 4. date 컬럼 추가 ======
def add_date_column(df):
    """
    seendate를 파싱하여 date(YYYYMMDD) 컬럼 추가
    
    Args:
        df: DataFrame
    
    Returns:
        pd.DataFrame
    """
    if df.empty:
        return df
    
    # seendate 형식: YYYYMMDDTHHMMSSZ 또는 다양한 형식
    def parse_seendate(s):
        try:
            # GDELT seendate는 보통 YYYYMMDDTHHMMSSZ 형식
            if 'T' in str(s):
                dt_str = str(s).split('T')[0]
                return dt_str
            else:
                # 그냥 숫자로 들어온 경우
                dt = datetime.strptime(str(s)[:8], '%Y%m%d')
                return dt.strftime('%Y%m%d')
        except:
            return None
    
    df['date'] = df['seendate'].apply(parse_seendate)
    df = df.dropna(subset=['date'])
    
    logger.info(f"date 컬럼 추가 완료: {len(df)}건")
    
    return df


# ====== 5. CSV 저장 ======
def save_to_csv(df, output_path):
    """
    DataFrame을 CSV로 저장
    
    Args:
        df: DataFrame
        output_path: 출력 파일 경로
    """
    if df.empty:
        logger.error("저장할 데이터가 없음")
        return
    
    # 디렉토리 생성
    output_dir = Path(output_path).parent
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # CSV 저장
    df.to_csv(output_path, index=False, encoding='utf-8')
    logger.info(f"저장 완료: {output_path} ({len(df)}건)")


# ====== 메인 실행 ======
if __name__ == "__main__":
    logger.info("=" * 60)
    logger.info("GDELT Bitcoin 기사 수집 + GKG Tone 조인 시작")
    logger.info(f"기간: {START_DATE} ~ {END_DATE}")
    logger.info("=" * 60)
    
    # 1. DOC API로 기사 수집
    logger.info("[1/4] GDELT DOC API로 Bitcoin 기사 수집 중...")
    articles = fetch_gdelt_doc_hourly(START_DATE, END_DATE, max_per_hour=MAX_ARTICLES_PER_HOUR)
    
    if not articles:
        logger.error("수집된 기사가 없습니다.")
    else:
        # 2. GKG Tone 매핑 구축
        logger.info("[2/4] GKG Tone 매핑 구축 중...")
        tone_mapping = build_tone_mapping(START_DATE, END_DATE)
        
        # 3. 조인 및 필터링
        logger.info("[3/4] 기사와 Tone 조인 중...")
        df = join_articles_with_tone(articles, tone_mapping)
        
        if not df.empty:
            # 4. date 컬럼 추가
            logger.info("[4/4] date 컬럼 추가 및 저장 중...")
            df = add_date_column(df)
            
            # 5. 저장
            save_to_csv(df, OUTPUT_PATH)
            
            logger.info("=" * 60)
            logger.info("완료!")
            logger.info(f"최종 데이터: {len(df)}건")
            logger.info(f"출력 파일: {OUTPUT_PATH}")
            logger.info("=" * 60)
        else:
            logger.error("Tone 매칭 후 데이터가 없습니다.")

2026-02-05 16:36:42,695 - INFO - ============================================================
2026-02-05 16:36:42,695 - INFO - GDELT Bitcoin 기사 수집 + GKG Tone 조인 시작
2026-02-05 16:36:42,695 - INFO - 기간: 2025-11-23 00:00:00+00:00 ~ 2026-01-14 23:59:59+00:00
2026-02-05 16:36:42,695 - INFO - ============================================================
2026-02-05 16:36:42,695 - INFO - [1/4] GDELT DOC API로 Bitcoin 기사 수집 중...
2026-02-05 16:36:45,208 - INFO - [20251123000000] 수집: 10건
2026-02-05 16:36:49,000 - INFO - [20251123010000] 수집: 10건
2026-02-05 16:36:51,774 - INFO - [20251123020000] 수집: 6건
2026-02-05 16:36:54,499 - INFO - [20251123030000] 수집: 10건
2026-02-05 16:36:58,194 - INFO - [20251123040000] 수집: 10건
2026-02-05 16:36:59,501 - INFO - [20251123050000] 수집: 8건
2026-02-05 16:37:02,335 - INFO - [20251123060000] 수집: 7건
2026-02-05 16:37:05,653 - INFO - [20251123070000] 수집: 10건
2026-02-05 16:37:09,368 - INFO - [20251123080000] 수집: 10건
2026-02-05 16:37:12,437 - INFO - [20251123090000] 수집: 9건
20

KeyboardInterrupt: 

## 셀 2: 일자별 Tone 집계 저장

**작업 내용**:
- `./data/raw/251123-260114-news.csv` 로드
- date(YYYYMMDD) 기준으로 tone_mean, tone_pos_share 평균 계산
- `./data/raw/251123-260114-tone.csv` 저장

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
일자별 Tone 집계
"""

import pandas as pd
from pathlib import Path

# ====== 설정 ======
INPUT_PATH = './data/raw/251123-260114-news.csv'
OUTPUT_PATH = './data/raw/251123-260114-tone.csv'

print("=" * 60)
print("일자별 Tone 집계 시작")
print("=" * 60)

# 1. CSV 로드
print(f"[1/3] 데이터 로드: {INPUT_PATH}")
df = pd.read_csv(INPUT_PATH, encoding='utf-8')
print(f"로드된 데이터: {len(df)}건")

# 2. date 컬럼 검증
print("[2/3] date 컬럼 검증")
if 'date' not in df.columns:
    raise ValueError("date 컬럼이 없습니다.")

# date를 문자열로 변환 (숫자로 읽힐 수 있음)
df['date'] = df['date'].astype(str)

# YYYYMMDD 형식 검증
invalid_dates = df[~df['date'].str.match(r'^\d{8}$')]
if not invalid_dates.empty:
    print(f"경고: {len(invalid_dates)}건의 잘못된 date 형식 발견")
    print(invalid_dates[['date']].head())

# 유효한 날짜만 유지
df = df[df['date'].str.match(r'^\d{8}$')]
print(f"유효한 데이터: {len(df)}건")

# 3. 일자별 집계
print("[3/3] 일자별 tone 집계")
tone_daily = df.groupby('date').agg({
    'tone_mean': 'mean',
    'tone_pos_share': 'mean'
}).reset_index()

tone_daily = tone_daily.sort_values('date')
print(f"집계 결과: {len(tone_daily)}일")

# 4. 저장
output_dir = Path(OUTPUT_PATH).parent
output_dir.mkdir(parents=True, exist_ok=True)

tone_daily.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"저장 완료: {OUTPUT_PATH}")

# 5. 결과 미리보기
print("\n[결과 미리보기]")
print(tone_daily.head(10))
print("\n[통계]")
print(tone_daily.describe())

print("=" * 60)
print("완료!")
print("=" * 60)

## 셀 3: BTC 종가 + Fear&Greed + Tone 병합 저장

**작업 내용**:
- Binance API로 BTC 일봉 종가(USD) 수집
- Alternative.me API로 Fear & Greed Index 수집
- Tone 데이터와 date 기준으로 병합
- `./data/raw/251123-260114-merged.csv` 저장
- 최종 컬럼: date, btc_price_usd, fear_greed, classification, tone_mean, tone_pos_share

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
BTC 종가 + Fear & Greed Index + Tone 병합
"""

import pandas as pd
import requests
from datetime import datetime, timezone
from pathlib import Path

# ====== 설정 ======
START_DATE = '20251123'
END_DATE = '20260114'
TONE_PATH = './data/raw/251123-260114-tone.csv'
OUTPUT_PATH = './data/raw/251123-260114-merged.csv'
TIMEOUT = 30

print("=" * 60)
print("BTC 종가 + Fear & Greed Index + Tone 병합")
print(f"기간: {START_DATE} ~ {END_DATE}")
print("=" * 60)

# ====== 1. BTC 일봉 종가 수집 (Binance API) ======
print("[1/5] BTC 일봉 종가 수집 (Binance API)")

def fetch_btc_price(start_date_str, end_date_str):
    """
    Binance API로 BTC 일봉 종가 수집
    """
    url = "https://api.binance.com/api/v3/klines"
    
    # YYYYMMDD를 datetime으로 변환
    start_dt = datetime.strptime(start_date_str, '%Y%m%d').replace(tzinfo=timezone.utc)
    end_dt = datetime.strptime(end_date_str, '%Y%m%d').replace(hour=23, minute=59, second=59, tzinfo=timezone.utc)
    
    start_time_ms = int(start_dt.timestamp() * 1000)
    end_time_ms = int(end_dt.timestamp() * 1000)
    
    params = {
        "symbol": "BTCUSDT",
        "interval": "1d",
        "startTime": start_time_ms,
        "endTime": end_time_ms,
        "limit": 1000
    }
    
    try:
        response = requests.get(url, params=params, timeout=TIMEOUT)
        response.raise_for_status()
        data = response.json()
        
        # Binance klines: [open_time, open, high, low, close, volume, ...]
        df = pd.DataFrame(data, columns=[
            'open_time', 'open', 'high', 'low', 'close', 'volume',
            'close_time', 'quote_volume', 'trades', 'taker_buy_base',
            'taker_buy_quote', 'ignore'
        ])
        
        # timestamp_ms와 종가 추출
        df['timestamp_ms'] = pd.to_numeric(df['open_time'])
        df['timestamp_utc'] = pd.to_datetime(df['timestamp_ms'], unit='ms', utc=True)
        df['date'] = df['timestamp_utc'].dt.strftime('%Y%m%d')
        df['btc_price_usd'] = pd.to_numeric(df['close'])
        
        df = df[['date', 'btc_price_usd']]
        print(f"BTC 가격 수집: {len(df)}일")
        
        return df
    
    except Exception as e:
        print(f"BTC 가격 수집 실패: {e}")
        return pd.DataFrame()

btc_df = fetch_btc_price(START_DATE, END_DATE)

# ====== 2. Fear & Greed Index 수집 ======
print("[2/5] Fear & Greed Index 수집")

def fetch_fear_greed_index():
    """
    Alternative.me API로 Fear & Greed Index 수집
    """
    url = "https://api.alternative.me/fng/"
    params = {
        "limit": 0,
        "format": "json"
    }
    
    try:
        response = requests.get(url, params=params, timeout=TIMEOUT)
        response.raise_for_status()
        data = response.json()
        
        if "data" not in data:
            print("API response missing 'data' field")
            return pd.DataFrame()
        
        df = pd.DataFrame(data["data"])
        
        # timestamp를 UTC datetime으로 변환
        df['timestamp'] = pd.to_numeric(df['timestamp'], errors='coerce')
        df['timestamp_utc'] = pd.to_datetime(df['timestamp'], unit='s', utc=True)
        df['date'] = df['timestamp_utc'].dt.strftime('%Y%m%d')
        
        # 필드명 매핑
        df = df.rename(columns={
            'value': 'fear_greed',
            'value_classification': 'classification'
        })
        
        df['fear_greed'] = pd.to_numeric(df['fear_greed'], errors='coerce').astype('Int64')
        
        # 기간 필터링
        df = df[(df['date'] >= START_DATE) & (df['date'] <= END_DATE)]
        
        # 같은 date에 여러 레코드가 있으면 가장 이른 것만 유지
        df = df.sort_values('timestamp_utc')
        df = df.drop_duplicates(subset=['date'], keep='first')
        
        df = df[['date', 'fear_greed', 'classification']]
        print(f"Fear & Greed Index 수집: {len(df)}일")
        
        return df
    
    except Exception as e:
        print(f"Fear & Greed Index 수집 실패: {e}")
        return pd.DataFrame()

fng_df = fetch_fear_greed_index()

# ====== 3. Tone 데이터 로드 ======
print(f"[3/5] Tone 데이터 로드: {TONE_PATH}")
tone_df = pd.read_csv(TONE_PATH, encoding='utf-8')
tone_df['date'] = tone_df['date'].astype(str)
print(f"Tone 데이터: {len(tone_df)}일")

# ====== 4. 데이터 병합 ======
print("[4/5] 데이터 병합 (date 기준)")

# BTC + Fear & Greed 병합
merged = pd.merge(btc_df, fng_df, on='date', how='inner')
print(f"BTC + Fear&Greed 병합: {len(merged)}일")

# Tone 데이터 병합
merged = pd.merge(merged, tone_df, on='date', how='inner')
print(f"최종 병합: {len(merged)}일")

# date 오름차순 정렬
merged = merged.sort_values('date').reset_index(drop=True)

# 컬럼 순서 조정
merged = merged[['date', 'btc_price_usd', 'fear_greed', 'classification', 'tone_mean', 'tone_pos_share']]

# ====== 5. 저장 ======
print(f"[5/5] 저장: {OUTPUT_PATH}")
output_dir = Path(OUTPUT_PATH).parent
output_dir.mkdir(parents=True, exist_ok=True)

merged.to_csv(OUTPUT_PATH, index=False, encoding='utf-8')
print(f"저장 완료: {len(merged)}건")

# ====== 결과 미리보기 ======
print("\n[결과 미리보기]")
print(merged.head(10))

print("\n[통계]")
print(merged.describe())

print("\n[날짜 누락 체크]")
all_dates = set(pd.date_range(
    start=datetime.strptime(START_DATE, '%Y%m%d'),
    end=datetime.strptime(END_DATE, '%Y%m%d'),
    freq='D'
).strftime('%Y%m%d'))
merged_dates = set(merged['date'])
missing_dates = sorted(all_dates - merged_dates)

if missing_dates:
    print(f"누락된 날짜 ({len(missing_dates)}일): {missing_dates[:10]}...")
else:
    print("누락된 날짜 없음")

print("=" * 60)
print("완료!")
print(f"출력 파일: {OUTPUT_PATH}")
print("=" * 60)